In [ ]:
import os
import asyncio
from nano_graphrag import GraphRAG
import networkx as nx

# ✅ STEP 1 – Carica il testo da un file
with open("../data/alleanze.txt", "r", encoding="utf-8") as f:
    text = f.read()

# ✅ STEP 2 – Imposta la tua chiave API OpenAI
os.environ["OPENAI_API_KEY"] ="YOUR_API_KEY"  # Sostituisci con la tua chiave reale

# ✅ STEP 3 – Inizializza GraphRAG
graphrag = GraphRAG(
    working_dir="./nano_cache",
    enable_local=True,
    tokenizer_type="tiktoken",
    tiktoken_model_name="gpt-3.5",  # usa gpt-4o solo se abilitato nel tuo account
    # graph_cluster_algorithm=None  # evitiamo errore sul clustering
)

# ✅ STEP 4 – Inserisci il documento (solo in ambienti Jupyter: usa `await`)
await graphrag.ainsert(text)

# ✅ STEP 5 – Carica il grafo salvato da file
G = nx.read_graphml("./nano_cache/graph_chunk_entity_relation.graphml")

# ✅ STEP 6 – Stampa le triplette soggetto --[relazione]--> oggetto
print("\n📌 Triplette estratte:")
for u, v, data in G.edges(data=True):
    relation = data.get("description", "(nessuna relazione)")
    print(f"{u} --[{relation}]--> {v}")



INFO:nano-graphrag:Loading tokenizer: type='tiktoken', name='gpt-3.5'
INFO:nano-graphrag:Load KV full_docs with 1 data
INFO:nano-graphrag:Load KV text_chunks with 1 data
INFO:nano-graphrag:Load KV llm_response_cache with 6 data
INFO:nano-graphrag:Load KV community_reports with 0 data
INFO:nano-graphrag:Loaded graph from ./nano_cache/graph_chunk_entity_relation.graphml with 53 nodes, 29 edges
INFO:nano-vectordb:Load (52, 1536) data
INFO:nano-vectordb:Init {'embedding_dim': 1536, 'metric': 'cosine', 'storage_file': './nano_cache/vdb_entities.json'} 52 data
INFO:nano-graphrag:[New Docs] inserting 1 docs
INFO:nano-graphrag:[New Chunks] inserting 1 chunks
INFO:nano-graphrag:[Entity Extraction]...


INFO:nano-graphrag:Inserting 44 vectors to entities


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:nano-graphrag:[Community Report]...
INFO:matplotlib.font_manager:generated new fontManager
INFO:nano-graphrag:Each level has communities: {0: 2}
INFO:nano-graphrag:Generating by levels: [0]
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:nano-graphrag:JSON data successfully extracted.


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:nano-graphrag:JSON data successfully extracted.


INFO:nano-graphrag:Writing graph with 53 nodes, 29 edges




📌 Triplette estratte:
"LA FRANCIA" --["La Francia and Il Regno Unito formed an alliance during World War II to combat shared enemies."]--> "IL REGNO UNITO"
"LA FRANCIA" --["La Francia and Gli Stati Uniti were allies during World War II."]--> "GLI STATI UNITI"
"LA FRANCIA" --["La Francia was part of Le forze alleate working together against the Axis powers."]--> "LE FORZE ALLEATE"
"IL REGNO UNITO" --["Il Regno Unito and Gli Stati Uniti were allies, collaborating during World War II."]--> "GLI STATI UNITI"
"IL REGNO UNITO" --["Il Regno Unito was part of Le forze alleate, collaborating with other Allied countries during World War II."]--> "LE FORZE ALLEATE"
"LA SECONDA GUERRA MONDIALE" --["La Germania was a major opposing force to the Allies during World War II."]--> "LA GERMANIA"
"LA SECONDA GUERRA MONDIALE" --["L'Unione Sovietica played a key role in the Allies' victory over Nazi Germany in World War II."]--> "L'UNIONE SOVIETICA"
"GLI STATI UNITI" --["Gli Stati Uniti were members of L

In [4]:
from neo4j import GraphDatabase
import networkx as nx

# ✅ Carica grafo da GraphML generato da GraphRAG
G = nx.read_graphml("./nano_cache/graph_chunk_entity_relation.graphml")

# ✅ Estrai le triplette (soggetto, relazione, oggetto)
triples = []
for u, v, data in G.edges(data=True):
    subject = u.replace('"', '')  # rimuove virgolette doppie
    obj = v.replace('"', '')
    relation = data.get("description", "relazione_non_specificata").replace('"', '')
    triples.append((subject, relation, obj))

# ✅ Connessione a Memgraph
driver = GraphDatabase.driver("bolt://localhost:7687", auth=("", ""))  # Memgraph non richiede autenticazione

# ✅ Inserimento delle triplette in Memgraph
with driver.session() as session:
    for s, r, o in triples:
        session.run("""
            MERGE (a:Entity {name: $s})
            MERGE (b:Entity {name: $o})
            MERGE (a)-[:RELATION {description: $r}]->(b)
        """, s=s, o=o, r=r)

driver.close()
print("✅ Triplette inserite in Memgraph.")

✅ Triplette inserite in Memgraph.


In [6]:
from neo4j import GraphDatabase

# ✅ Connessione a Memgraph
driver = GraphDatabase.driver("bolt://localhost:7687", auth=("", ""))

# ✅ Query per vedere i dati
with driver.session() as session:
    result = session.run("MATCH (a)-[r]->(b) RETURN a.name, r.description, b.name LIMIT 10")
    for record in result:
        print(f"{record['a.name']} --[{record['r.description']}]--> {record['b.name']}")

driver.close()

LA FRANCIA --[La Francia and Il Regno Unito formed an alliance during World War II to combat shared enemies.]--> IL REGNO UNITO
LA FRANCIA --[La Francia and Gli Stati Uniti were allies during World War II.]--> GLI STATI UNITI
IL REGNO UNITO --[Il Regno Unito and Gli Stati Uniti were allies, collaborating during World War II.]--> GLI STATI UNITI
LA FRANCIA --[La Francia was part of Le forze alleate working together against the Axis powers.]--> LE FORZE ALLEATE
IL REGNO UNITO --[Il Regno Unito was part of Le forze alleate, collaborating with other Allied countries during World War II.]--> LE FORZE ALLEATE
GLI STATI UNITI --[Gli Stati Uniti were members of Le forze alleate, contributing to the collective efforts in World War II.]--> LE FORZE ALLEATE
LA GERMANIA --[Le forze alleate opposed La Germania during World War II, combating its military aggression across Europe.]--> LE FORZE ALLEATE
L'UNIONE SOVIETICA --[L'Unione Sovietica joined Le forze alleate to fight against Nazi Germany and i

In [ ]:
from neo4j import GraphDatabase
from openai import OpenAI
import os
from dotenv import load_dotenv

load_dotenv()

# ✅ Nuova API OpenAI
client = OpenAI(api_key=os.getenv("YOUR_API_KEY"))

# ✅ Connessione a Memgraph
driver = GraphDatabase.driver("bolt://localhost:7687", auth=("", ""))

# ✅ Query per ottenere i dati
with driver.session() as session:
    result = session.run("""
        MATCH (a)-[r]->(b) 
        RETURN a.name, r.description, b.name
    """)
    
    context = "\n".join([f"{record['a.name']} {record['r.description']} {record['b.name']}" 
                        for record in result])

driver.close()

# ✅ Query con nuova API OpenAI
response = client.chat.completions.create(
    model="gpt-4",
    messages=[
        {"role": "system", "content": "Rispondi solo in base al contesto fornito."},
        {"role": "user", "content": f"Contesto:\n{context}\n\nDomanda: Quali alleanze ha stretto la Francia?"}
    ]
)

print("Risposta LLM:")
print(response.choices[0].message.content)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Risposta LLM:
La Francia ha stretto un'alleanza con Il Regno Unito e Gli Stati Uniti durante la Seconda Guerra Mondiale. Inoltre, la Francia faceva parte delle forze alleate che includevano molte nazioni unite contro le potenze dell'Asse.
